# US EPA Air Quality System API Example
This illustrates how to request data from the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. This is a historical API and does not provide real-time air quality data. The [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

This notebook works systematically through example calls, requesting an API key, using 'list' to get various IDs and parameter values, and using 'daily summary' to get summary data that meets specific condistions. The notebook contains example function definitions that could be reused in other code. In general, the notebook explains each step along the way, referring back to possible output. Some of the explanations are tailored to the specific example requests of the API. Changing values to explore the results of the API is probably useful, but that will result in some explanations being out of sync with the outputs.

The US EPA was created in the early 1970's. The EPA reports that they only started broad based monitoring with standardized quality assurance procedures in the 1980's. Many counties will have data starting somewhere between 1983 and 1988. However, some counties still do not have any air quality monitoring stations. The API helps resolve this by providing calls to search for monitoring stations and data using either station ids, or a county designation or a geographic bounding box. This example code provides examples of the county based and bounding box based API calls. Some [additional information on the Air Quality System can be found in the EPA FAQ](https://www.epa.gov/outdoor-air-quality-data/frequent-questions-about-airdata) on the system.

The end goal is to get to some values that we might use for the Air Quality Index or AQI. You might see this reported on the news, most often around smog, but more frequently with regard to smoke. The AQI index is meant to tell us something about how healthy or clean the air is on any day. The AQI is actually a somewhat complext measure. When I started this example I looked up [how to calculate the AQI](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf) so that I would know roughly what goes into that value.


## License
This code was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - September 5, 2023.

In [1]:
# These are standard python modules
import json, time
#
#    The 'requests' module is a distribution module for making web requests.
#
import requests
import pandas as pd

In [2]:
#########
#
#    CONSTANTS
#

#
#    This is the root of all AQS API URLs
#
API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

#
#    These are 'actions' we can ask the API to take or requests that we can make of the API
#
#    Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'
#
#    List actions provide information on API parameter values that are required by some other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'
#
#    Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '/monitors/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_MONITORS_BOX = '/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '/dailyData/byCounty?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&state={state}&county={county}'
API_ACTION_DAILY_SUMMARY_BOX = '/dailyData/byBox?email={email}&key={key}&param={param}&bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'
#
#    It is always nice to be respectful of a free data resource.
#    We're going to observe a 100 requests per minute limit - which is fairly nice
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED
#
#
#    This is a template that covers most of the parameters for the actions we might take, from the set of actions
#    above. In the examples below, most of the time parameters can either be supplied as individual values to a
#    function - or they can be set in a copy of the template and passed in with the template.
# 
AQS_REQUEST_TEMPLATE = {
    "email":      "",     
    "key":        "",      
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format, begin_date and end_date must be in the same year
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",     # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""      # parameter class is only used by the List calls
}



I will need to request an API key to access the API. I should test access to the API to understand whether I can get data for my city based on the US County where my city is located, or whether I need to create a geodetic bounding box to access station data.

## Making a sign-up request

Before we can use the API, we need to request a key. We will use an email address to make the request. The EPA then sends a confirmation email link and a 'key' that we use for all other requests.

We only need to sign-up once, unless we want to invalidate our current key (by getting a new key) or we lose our key.

In [3]:
#
#    This implements the sign-up request. The parameters are standardized so that this function definition matches
#    all of the others. However, the easiest way to call this is to simply call this function with your preferred
#    email address.
#
def request_signup(email_address = None,
                   endpoint_url = API_REQUEST_URL, 
                   endpoint_action = API_ACTION_SIGNUP, 
                   request_template = AQS_REQUEST_TEMPLATE,
                   headers = None):
    
    # Make sure we have a string - if you don't have access to this email addres, things might go badly for you
    if email_address:
        request_template['email'] = email_address        
    if not request_template['email']: 
        raise Exception("Must supply an email address to call 'request_signup()'")
    
    # Compose the signup url - create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [4]:
#
#    A SIGNUP request is only to be done once, to request a key. A key is sent to that email address and needs to be confirmed with a click through
#    This code should probably be commented out after you've made your key request to make sure you don't accidentally make a new sign-up request
#
#print("Requesting SIGNUP ...")
#response = request_signup("sagnik99@uw.edu")
#print(json.dumps(response,indent=4))
#

In [5]:
USERNAME = "sagnik99@uw.edu"
APIKEY= 'tealgoose54'

## Making a list request
Once you have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. The one problem here is that some of the responses rely on jargon or terms-of-art. That is, one needs to know a bit about the way atmospheric sciece works to understand some of the terms. ... Good thing we can use the web to search for terms we don't know!

In [6]:
#
#    This implements the list request. There are several versions of the list request that only require email and key.
#    This code sets the default action/requests to list the groups or parameter class descriptors. Having those descriptors 
#    allows one to request the individual (proprietary) 5 digit codes for individual air quality measures by using the
#    param request. Some code in later cells will illustrate those requests.
#
def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



In [7]:
#
#   The default should get us a list of the various groups or classes of sensors. These classes are user defined names for clustors of
#   sensors that might be part of a package or default air quality sensing station. We need a class name to start getting down to the
#   a sensor ID. Each sensor type has an ID number. We'll eventually need those ID numbers to be able to request values that come from
#   that specific sensor.
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

We're interested in getting to something that might be the Air Quality Index (AQI). You see this reported on the news - often around smog values, but also when there is smoke in the sky. The AQI is a complex measure of different gasses and of the particles in the air (dust, dirt, ash ...).

From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.


In [8]:
#
#   Once we have a list of the classes or groups of possible sensors, we can find the sensor IDs that make up that class (group)
#   The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
#   We'll use that to make another list request.
#
AQI_PARAM_CLASS = "AQI POLLUTANTS"

In [9]:
#
#   Structure a request to get the sensor IDs associated with the AQI
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['pclass'] = AQI_PARAM_CLASS  # here we specify that we want this 'pclass' or parameter classs

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))


[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We should now have (above) a response containing a set of sensor ID numbers. The list should include the sensor numbers as well as a description or name for each sensor. 

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up.

What I did below was to break the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [10]:
# Given the set of sensor codes, now we can create a parameter list or 'param' value as defined by the AQS API spec.
# It turns out that we want all of these measures for AQI, but we need to have two different param constants to get
# all seven of the code types. We can only have a max of 5 sensors/values request per param.

#   Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"
#   Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"

Air quality monitoring stations are located all over the US at different locations. We will need some sample locations to experiment with different locations to see what kinds of values come back from different sensor requests.

This list includes the [FIPS](https://www.census.gov/library/reference/code-lists/ansi.html) number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.

Used Tulare as assigned. 

In [11]:
# We'll use these two city locations in the examples below.
CITY_LOCATIONS = {
    'tulare' :       {'city'   : 'Tulare',
                       'county' : 'Tulare',
                       'state'  : 'California',
                       'fips'   : '06107',
                       'latlon' : [36.2714, -118.7770] }
}

Given our CITY_LOCATIONS constant we can now find which monitoring locations are nearby. One option is to use the county to define the area we're interest in. You can get the EPA to list their monitoring stations by county. You can also get a set of monitoring stations by using a bounding box of latitude, longitude points. For this example, we'll use the county approach. There is a bounding box example later in this notebook.

In [12]:
#
#  This list request should give us a list of all the monitoring stations in the county specified by the
#  given city selected from the CITY_LOCATIONS dictionary
#
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY_LOCATIONS['tulare']['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY_LOCATIONS['tulare']['fips'][2:]  # the last three digits (characters) of FIPS is the county code

response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": "Sequoia NP-Lower Kaweah"
    },
    {
        "code": "0003",
        "value_represented": "Sequoia NP-Lodgepole Chlorinator"
    },
    {
        "code": "0005",
        "value_represented": "Sequoia & Kings Canyon NPs - Ash Mountain"
    },
    {
        "code": "0006",
        "value_represented": "Sequoia & Kings Canyon NPs - Lower Kaweah"
    },
    {
        "code": "0007",
        "value_represented": "Sequoia & Kings Canyon NPs - Grant Grove"
    },
    {
        "code": "0008",
        "value_represented": "Sequoia NP-Lookout Point Ranger Station"
    },
    {
        "code": "0009",
        "value_represented": "Sequoia & Kings Canyon NPs - Ash Mountain"
    },
    {
        "code": "1001",
        "value_represented": "Sequoia NP-Ash Mountain"
    },
    {
        "code": "1002",
        "value_represented": null
    },
    {
        "code": "1

The above response gives us a list of monitoring stations. Each monitoring station has a unique "code" which is a string number, and, sometimes, a description. The description seems to be something about where the monitoring station is located.

## Making a daily summary request

The function below is designed to encapsulate requests to the EPA AQS API. When calling the function one should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call simply pass in the parameters that need to change, like date ranges.

Another function below provides an example of extracting values and restructuring the response to make it a little more usable.

In [13]:
#
#    This implements the daily summary request. Daily summary provides a daily summary value for each sensor being requested
#    from the start date to the end date. 
#
#    Like the two other functions, this can be called with a mixture of a defined parameter dictionary, or with function
#    parameters. If function parameters are provided, those take precedence over any parameters from the request template.
#
def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers
        
    # compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response



The form of the daily summary response is a bit verbose with lots of repeated values. What we'll do is create a data structure that relies on a hierarchical context to summarize the data.

The two responses show that not every monitoring site produces values. As well, it looks like the monitoring sites only produce values for particulates and not for gaseous pollutants.

The next function takes the response and a set of fields that should be extracted for their data values. The code assumes those fields are available. If there are missing values something could certainly go wrong. The function creates a summary for each monitoring site.

In [14]:
#
#    This is a list of field names - data - that will be extracted from each record
#
EXTRACTION_FIELDS = ['sample_duration','observation_count','arithmetic_mean','aqi']

#
#    The function creates a summary record
def extract_summary_from_response(r=None, fields=EXTRACTION_FIELDS):
    ## the result will be structured around monitoring site, parameter, and then date
    result = dict()
    data = r["Data"]
    for record in data:
        # make sure the record is set up
        site = record['site_number']
        param = record['parameter_code']
        #date = record['date_local']    # this version keeps the respnse value YYYY-
        date = record['date_local'].replace('-','') # this puts it in YYYYMMDD format
        if site not in result:
            result[site] = dict()
            result[site]['local_site_name'] = record['local_site_name']
            result[site]['site_address'] = record['site_address']
            result[site]['state'] = record['state']
            result[site]['county'] = record['county']
            result[site]['city'] = record['city']
            result[site]['pollutant_type'] = dict()
        if param not in result[site]['pollutant_type']:
            result[site]['pollutant_type'][param] = dict()
            result[site]['pollutant_type'][param]['parameter_name'] = record['parameter']
            result[site]['pollutant_type'][param]['units_of_measure'] = record['units_of_measure']
            result[site]['pollutant_type'][param]['method'] = record['method']
            result[site]['pollutant_type'][param]['data'] = dict()
        if date not in result[site]['pollutant_type'][param]['data']:
            result[site]['pollutant_type'][param]['data'][date] = list()
        
        # now extract the specified fields
        extract = dict()
        for k in fields:
            if str(k) in record:
                extract[str(k)] = record[k]
            else:
                # this makes sure we always have the requested fields, even if
                # we have a missing value for a given day/month
                extract[str(k)] = None
        
        # add this extraction to the list for the day
        result[site]['pollutant_type'][param]['data'][date].append(extract)
    
    return result


The following cell uses the functions defined above and extracts data from 1963 to 2023. A loop is used as during one API call only 1 years of data can be accessed

In [15]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_PARTICULATES
request_data['state'] = CITY_LOCATIONS['tulare']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['tulare']['fips'][2:]

# Initialize an empty list to store the results
all_particulate_data = []

# Define the request template and base URL
request_template = "YOUR_REQUEST_TEMPLATE_HERE"
base_url = "YOUR_BASE_URL_HERE"

# Define the years you want to retrieve data for
start_year = 1963
end_year = 2023

# Loop through the years and request daily summary data
for year in range(start_year, end_year + 1):
    begin_date = f"{year}0101"
    end_date = f"{year}1231"

    # Make the request for the current year
    particulate_aqi = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)
    
    # Check the response and append data to the list
    if particulate_aqi["Header"][0]['status'] == "Success":
        #all_gaseous_data.extend(gaseous_aqi['Data'])
        extract_particulate = extract_summary_from_response(particulate_aqi)
        #print("Summary of particulate extraction ...")
        #print(json.dumps(extract_gaseous,indent=4))
        all_particulate_data.append(extract_particulate)
        
    elif particulate_aqi["Header"][0]['status'].startswith("No data "):
        print(f"No data available for {year}.")
        
    else:
        print(f"Error in retrieving data for {year}.")

# # Print or process the collected data
# print("Response for the particulate pollutants ...")
# print(json.dumps(all_particulate_data, indent=4))

# Now, the all_gaseous_data list contains data for each year from 1963 to 2023.


No data available for 1963.
No data available for 1964.
No data available for 1965.
No data available for 1966.
No data available for 1967.
No data available for 1968.
No data available for 1969.
No data available for 1970.
No data available for 1971.
No data available for 1972.
No data available for 1973.
No data available for 1974.
No data available for 1975.
No data available for 1976.
No data available for 1977.
No data available for 1978.
No data available for 1979.
No data available for 1980.
No data available for 1981.
No data available for 1982.
No data available for 1983.
No data available for 1984.
No data available for 1985.


The following cell takes our raw data and stores it as a JSON file in our computer for later use. 

In [16]:
# Define the output file name
output_file = "../intermediate data/particulate_data.json"

# Write the JSON data to the text file with an indentation of 4
with open(output_file, 'w') as file:
    json.dump(all_particulate_data, file, indent=4)

print(f"JSON data has been written to {output_file} with an indentation of 4.")

JSON data has been written to ../intermediate data/particulate_data.json with an indentation of 4.


In the below two cells the same work is being done as in the above two cells. First aqi data is extracted for gaseous for 1963 to 2023 and then raw file is stored

In [17]:
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['param'] = AQI_PARAMS_GASEOUS
request_data['state'] = CITY_LOCATIONS['tulare']['fips'][:2]
request_data['county'] = CITY_LOCATIONS['tulare']['fips'][2:]

# Initialize an empty list to store the results
all_gaseous_data = []

# Define the request template and base URL
request_template = "YOUR_REQUEST_TEMPLATE_HERE"
base_url = "YOUR_BASE_URL_HERE"

# Define the years you want to retrieve data for
start_year = 1963
end_year = 2023

# Loop through the years and request daily summary data
for year in range(start_year, end_year + 1):
    begin_date = f"{year}0101"
    end_date = f"{year}1231"

    # Make the request for the current year
    gaseous_aqi = request_daily_summary(request_template=request_data, begin_date=begin_date, end_date=end_date)
    
    # Check the response and append data to the list
    if gaseous_aqi["Header"][0]['status'] == "Success":
        #all_gaseous_data.extend(gaseous_aqi['Data'])
        extract_gaseous = extract_summary_from_response(gaseous_aqi)
        #print("Summary of particulate extraction ...")
        #print(json.dumps(extract_gaseous,indent=4))
        all_gaseous_data.append(extract_gaseous)
        
    elif gaseous_aqi["Header"][0]['status'].startswith("No data "):
        print(f"No data available for {year}.")
        
    else:
        print(f"Error in retrieving data for {year}.")

# # Print or process the collected data
# print("Response for the gaseous pollutants ...")
# print(json.dumps(all_gaseous_data, indent=4))

# Now, the all_gaseous_data list contains data for each year from 1963 to 2023.


No data available for 1963.
No data available for 1964.
No data available for 1965.
No data available for 1966.
No data available for 1967.
No data available for 1968.
No data available for 1969.


In [18]:
# Define the output file name
output_file = "../intermediate data/gaseous_data.json"

# Write the JSON data to the text file with an indentation of 4
with open(output_file, 'w') as file:
    json.dump(all_gaseous_data, file, indent=4)

print(f"JSON data has been written to {output_file} with an indentation of 4.")

JSON data has been written to ../intermediate data/gaseous_data.json with an indentation of 4.


The below function takes as input the raw aqi list of dicts and returns processed output with aqi, sample_duration and the date for which it was taken. 

After this extract_aqi() is used on both gaseous and particulate data. 

Once the data is extracted it is further filtered down and converted to the dataframe. 

In [19]:
def extract_aqi(data, dynamic_keys=None):
    if dynamic_keys is None:
        dynamic_keys = []

    extracted_data = []

    for key, value in data.items():
        current_keys = dynamic_keys + [key]

        if isinstance(value, dict):
            extracted_data.extend(extract_aqi(value, current_keys))
        elif isinstance(value, list):
            for item in value:
                if isinstance(item, dict):
                    aqi = item.get("aqi")
                    sample_duration = item.get("sample_duration")
                    if aqi is not None:
                        extracted_data.append({"keys": current_keys,"sample_duration": sample_duration, "aqi": aqi})

    return extracted_data

In [20]:
# Initialize a list to store the extracted data
extracted_data_gaseous = []

# Iterate through the list of datasets
for dataset in all_gaseous_data:
    extracted_data_gaseous.extend(extract_aqi(dataset))

# Print or further process the extracted data
for entry in extracted_data_gaseous:
    keys = " > ".join(entry["keys"])
    aqi = entry["aqi"]
    sample = entry['sample_duration']
#     print(f"Keys: {keys}, AQI: {aqi} , SAMPLE: {sample}")

In [21]:
# Initialize a list to store the extracted data
extracted_data_particulate = []

# Iterate through the list of datasets
for dataset in all_particulate_data:
    extracted_data_particulate.extend(extract_aqi(dataset))

# Print or further process the extracted data
for entry in extracted_data_particulate:
    keys = " > ".join(entry["keys"])
    aqi = entry["aqi"]
    sample = entry['sample_duration']
#     print(f"Keys: {keys}, AQI: {aqi} , SAMPLE: {sample}")

In [22]:
# extracted_data_particulate

In [23]:
# Extract relevant information and create a new list of dictionaries
formatted_data = []

for entry in extracted_data_particulate:
    keys = entry['keys']
    date = keys[-1]
    aqi = entry['aqi']
    sample_duration = entry['sample_duration']
    pollutant_type = keys[-3]

    formatted_data.append({'Date': date, 'AQI': aqi, 'Sample_Duration': sample_duration, 'Pollutant_Type': pollutant_type})

# Create a DataFrame from the formatted data
particulate_df = pd.DataFrame(formatted_data)

# Convert the 'Date' column to a datetime format
particulate_df['Date'] = pd.to_datetime(particulate_df['Date'], format='%Y%m%d')

In [24]:
particulate_df.head()

,Date,AQI,Sample_Duration,Pollutant_Type
0,1986-11-04,87,24 HOUR,81102
1,1986-11-10,79,24 HOUR,81102
2,1986-11-16,87,24 HOUR,81102
3,1986-11-22,57,24 HOUR,81102
4,1986-11-28,69,24 HOUR,81102


In [25]:
# Extract relevant information and create a new list of dictionaries
formatted_data = []

for entry in extracted_data_gaseous:
    keys = entry['keys']
    date = keys[-1]
    aqi = entry['aqi']
    sample_duration = entry['sample_duration']
    pollutant_type = keys[-3]

    formatted_data.append({'Date': date, 'AQI': aqi, 'Sample_Duration': sample_duration, 'Pollutant_Type': pollutant_type})

# Create a DataFrame from the formatted data
gaseous_df = pd.DataFrame(formatted_data)

# Convert the 'Date' column to a datetime format
gaseous_df['Date'] = pd.to_datetime(gaseous_df['Date'], format='%Y%m%d')

In [26]:
gaseous_df.head()

,Date,AQI,Sample_Duration,Pollutant_Type
0,1970-03-19,38,1 HOUR,42602
1,1970-03-19,38,1 HOUR,42602
2,1970-03-18,47,1 HOUR,42602
3,1970-03-18,47,1 HOUR,42602
4,1970-03-20,57,1 HOUR,42602


In [27]:
# Both the dataframes are saved so thtey can be used later easily and time can be saved
gaseous_df.to_csv("../intermediate data/gaseous_aqi_data_processed.csv")
particulate_df.to_csv("../intermediate data/particulate_aqi_data_processed.csv")

In the following code, the primary objective is to combine and analyze air quality data from two separate DataFrames, particulate_df and gaseous_df. The code begins by creating copies of these DataFrames and removing certain columns that are not required for the analysis. Next, the code groups the data by date, calculates the average air quality measurements for each date, and combines the results into a single DataFrame. The code then computes a unified Air Quality Index (AQI) from the individual gaseous and particulate AQI values, filling in missing values as needed. Afterward, the code extracts the year from the 'Date' column, groups the data by year, and calculates the mean AQI for each year. The result is stored in the aqi_df DataFrame, providing a summary of annual average AQI values for further analysis or visualization.

In [28]:
particulate_df1 = particulate_df.copy()
gaseous_df1 = gaseous_df.copy()

In [29]:
particulate_df1 = particulate_df1.drop(['Sample_Duration', 'Pollutant_Type'], axis=1)
gaseous_df1 = gaseous_df1.drop(['Sample_Duration', 'Pollutant_Type'], axis=1)

In [30]:
# Group by 'Date' and calculate the mean for each group
particulate_avg_df = particulate_df1.groupby('Date', as_index=False).mean()
gaseous_avg_df = gaseous_df1.groupby('Date', as_index=False).mean()

In [31]:
# Merge the DataFrames on 'Date' and take the maximum AQI or fill with values from either DataFrame
combined_df = gaseous_avg_df.merge(particulate_avg_df, on='Date', how='outer', suffixes=('_gaseous', '_particulate'))
combined_df['AQI'] = combined_df[['AQI_gaseous', 'AQI_particulate']].max(axis=1)
combined_df['AQI'].fillna(combined_df['AQI_gaseous'].combine_first(combined_df['AQI_particulate']), inplace=True)
combined_df.drop(columns=['AQI_gaseous', 'AQI_particulate'], inplace=True)
# Extract the year from the 'Date' column
combined_df['Year'] = combined_df['Date'].dt.year

In [32]:
# Group by 'Year' and calculate the mean AQI for each year
aqi_df = combined_df.groupby('Year')['AQI'].mean().reset_index()

In [33]:
# Displaying what aqi_df looks like
aqi_df.head()

,Year,AQI
0,1970,41.989655
1,1971,47.299543
2,1972,39.757741
3,1973,43.811872
4,1974,40.801826


In [34]:
# Save the aqi_df dataframe
aqi_df.to_csv("../intermediate data/final_aqi_each_year.csv")